In [2]:
! pip install transformers
! pip install torch
! pip install pandas
! pip install openai
! pip install python-dotenv

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/67/78/7588a047e458cb8075a4089d721d7af5e143ff85a2388d4a28c530be0494/openai-0.27.8-py3-none-any.whl.metadata
  Using cached openai-0.27.8-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [3]:
import os
import openai
import torch
import pandas as pd
from dotenv import load_dotenv, dotenv_values
from transformers import AutoTokenizer, AutoModelForSequenceClassification

load_dotenv()

/home/mpcen/workspace/projects/web2/1es-hackathon/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
openai.api_type = "azure"
openai.api_base = "https://mpcen-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
system_prompt = [
    {
        "role": "system",
        "content": "You are an expert that specializes in open source licenses. You will respond to questions about open source licenses. Your response will be 1 word, the license in SPDX format. Terminate the session after your response"
    }
]

user_prompt = [
    {
        "role": "user",
        "content": ""
    }
]

In [6]:
def get_license_chatgpt(text):
  user_prompt[0]['content'] = text
  response = openai.ChatCompletion.create(
    engine="mpcen-gpt-35-turbo-0301",
    messages = system_prompt + user_prompt,
    temperature=0,
    max_tokens=350,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)
  return response.choices[0].message.content

In [7]:
tokenizer = AutoTokenizer.from_pretrained("mpcen/legal-001")
model = AutoModelForSequenceClassification.from_pretrained("mpcen/legal-001")

In [8]:
labels_df = pd.read_json("./labels.json", typ="series")
trending_repos_df = pd.read_json("./trending-repos-licenses.json", typ="series")
# trending_repos_df = pd.read_json("./broken-licenses.json", typ="series")

In [9]:
def get_license_legal001(text):
    tokenized_input = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    output = model(**tokenized_input)
    prediction_value = torch.argmax(output.logits).item()
    return labels_df.loc[labels_df.index[prediction_value]]

In [10]:
for repo in trending_repos_df:
    text = repo["text"]
    actual_license = repo["license"]
    
    legal001_license = get_license_legal001(text)
    chatgpt35_license = get_license_chatgpt(text)
    
    print(f"-------------------")
    print(f"Actual license: {actual_license}")
    print(f"mpcen/legal-001 license: {legal001_license}")
    print(f"ChatGPT-3.5 license: {chatgpt35_license}")
    print(f"-------------------")
    print("")
    

-------------------
Actual license: Apache-2.0
mpcen/legal-001 license: Apache-2.0
ChatGPT-3.5 license: Apache-2.0
-------------------

-------------------
Actual license: MIT
mpcen/legal-001 license: MIT
ChatGPT-3.5 license: MIT
-------------------

-------------------
Actual license: AGPL-3.0
mpcen/legal-001 license: AGPL-3.0
ChatGPT-3.5 license: AGPL-3.0-or-later
-------------------

-------------------
Actual license: GPL-2.0
mpcen/legal-001 license: GPL-2.0
ChatGPT-3.5 license: GPL-2.0-only
-------------------

-------------------
Actual license: OFL-1.1
mpcen/legal-001 license: OFL-1.1
ChatGPT-3.5 license: OFL-1.1
-------------------

-------------------
Actual license: MPL-2.0
mpcen/legal-001 license: MPL-2.0
ChatGPT-3.5 license: MPL-2.0
-------------------

-------------------
Actual license: BSD-3-Clause
mpcen/legal-001 license: BSD-3-Clause
ChatGPT-3.5 license: BSD-3-Clause
-------------------

-------------------
Actual license: GPL-3.0
mpcen/legal-001 license: GPL-3.0-only
